In [1]:
from typing import Dict, Optional, Sequence, Tuple

from google.cloud import aiplatform
from google.cloud.aiplatform import explain

import tensorflow as tf

In [2]:
project_id='ihr-vertex-pipelines'
my_region='europe-west4' # :flag-nl:

In [ ]:
def upload_model(
    project: str,
    location: str,
    display_name: str,
    serving_container_image_uri: str,
    artifact_uri: Optional[str] = None,
    serving_container_predict_route: Optional[str] = None,
    serving_container_health_route: Optional[str] = None,
    description: Optional[str] = None,
    serving_container_command: Optional[Sequence[str]] = None,
    serving_container_args: Optional[Sequence[str]] = None,
    serving_container_environment_variables: Optional[Dict[str, str]] = None,
    serving_container_ports: Optional[Sequence[int]] = None,
    instance_schema_uri: Optional[str] = None,
    parameters_schema_uri: Optional[str] = None,
    prediction_schema_uri: Optional[str] = None,
    explanation_metadata: Optional[explain.ExplanationMetadata] = None,
    explanation_parameters: Optional[explain.ExplanationParameters] = None,
    sync: bool = True,
):

    aiplatform.init(project=project, location=location)

    model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        serving_container_predict_route=serving_container_predict_route,
        serving_container_health_route=serving_container_health_route,
        instance_schema_uri=instance_schema_uri,
        parameters_schema_uri=parameters_schema_uri,
        prediction_schema_uri=prediction_schema_uri,
        description=description,
        serving_container_command=serving_container_command,
        serving_container_args=serving_container_args,
        serving_container_environment_variables=serving_container_environment_variables,
        serving_container_ports=serving_container_ports,
        explanation_metadata=explanation_metadata,
        explanation_parameters=explanation_parameters,
        sync=sync,
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model


## Upload and deploy raw model

This is just a raw deploy of the `saved_model.pb`, so using this model requires using the transformed data in TF-IDF format (or whatever the model was trained with). What we want is a model that takes the text of a review. This requires:

* Including the `transform_fn` somehow
* Probably [specifying schemas](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models?&_ga=2.51426537.-1793164395.1642757755#predictschemata), predict route? Deploying a custom container?

In [ ]:
model = upload_model(
    project=project_id,
    location=my_region,
    display_name='my first model',
    serving_container_image_uri='europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest',
    artifact_uri='gs://ihr-vertex-pipelines/0.13+11.gbee0168/batch=8192/epochs=15/model/saved_model/'
)

Check out deployed models: [deployed models in console](https://pantheon2.corp.google.com/vertex-ai/models?project=ihr-vertex-pipelines)

In [ ]:
def create_endpoint(
    project: str, display_name: str, location: str,
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name, project=project, location=my_region,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint



In [ ]:
endpoint = create_endpoint(project_id, 'raw_model_endpoint', my_region)
endpoint_id = endpoint.resource_name.split('/')[-1]

Check out model endpoints: [model endpoints in console](https://pantheon2.corp.google.com/vertex-ai/endpoints?project=ihr-vertex-pipelines)

In [ ]:
endpoint_id

In [ ]:
def deploy_model_with_dedicated_resources(
    project,
    location,
    model_name: str,
    machine_type: str,
    endpoint: Optional[aiplatform.Endpoint] = None,
    deployed_model_display_name: Optional[str] = None,
    traffic_percentage: Optional[int] = 0,
    traffic_split: Optional[Dict[str, int]] = None,
    min_replica_count: int = 1,
    max_replica_count: int = 1,
    accelerator_type: Optional[str] = None,
    accelerator_count: Optional[int] = None,
    explanation_metadata: Optional[explain.ExplanationMetadata] = None,
    explanation_parameters: Optional[explain.ExplanationParameters] = None,
    metadata: Optional[Sequence[Tuple[str, str]]] = (),
    sync: bool = True,
):
    """
        model_name: A fully-qualified model resource name or model ID.
              Example: "projects/123/locations/us-central1/models/456" or
              "456" when project and location are initialized or passed.
    """

    aiplatform.init(project=project, location=location)

    model = aiplatform.Model(model_name=model_name)

    # The explanation_metadata and explanation_parameters should only be
    # provided for a custom trained model and not an AutoML model.
    model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        traffic_percentage=traffic_percentage,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        explanation_metadata=explanation_metadata,
        explanation_parameters=explanation_parameters,
        metadata=metadata,
        sync=sync, # Whether to execute this method synchronously
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model


In [ ]:
deployed_model = deploy_model_with_dedicated_resources(project_id, my_region, model.resource_name, 'n1-standard-4', endpoint, traffic_percentage=100)

## Get some predictions from the deployed model using test data

In [63]:
def endpoint_predict(
    project: str, location: str, instances: list, endpoint: str
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    print(prediction)
    return prediction

In [3]:
testdata = 'gs://ihr-vertex-pipelines/data/prepared/test/test_data-00000-of-00001.tfrecord'
raw_dataset = tf.data.TFRecordDataset([testdata])
raw_dataset

2022-01-25 12:27:49.914697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-01-25 12:27:49.914735: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-25 12:27:49.914768: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vm-7f3e4f9c-1f26-48e8-9195-c899f55e78bc): /proc/driver/nvidia/version does not exist
2022-01-25 12:27:49.915091: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [12]:
def parse_tf_record(record):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  return example

raw_instances = [parse_tf_record(raw_record) for raw_record in raw_dataset.take(3)]

In [52]:
raw_instances[0].features.feature['text'].bytes_list.value[0]

b'************* SPOILERS BELOW ************* "\'Night, Mother" is the story of Jesse (Sissy Spacek), a divorced epileptic woman who calmly announces to her brash mother (Anne Bancroft) that she\'s going to commit suicide. This is a fascinating premise that is drained of all vitality and excitement. The brilliant hook turns out to be a cheat- the story that follows is lacking in substance, gravity and revelatory value. Where are the shocks and surprises as mother and daughter have what may be the last conversation of their lives? Where are the secrets revealed, the confessions and fantasies and regrets? They\'re here, but they\'ve all been painted the same dull color that keeps emotion in the background and celebrates the \'genius\' of playwright Marsha Norman at the expense of everything else. The result is not a film but an exhausting endurance test.<br /><br />Let me preface my comments by saying I find Sissy Spacek to be one of the greatest actresses in the history of motion picture

In [56]:
def to_instance(raw_record, key):
    text_bytes = raw_record.features.feature['text'].bytes_list.value[0]
    return { 'values': text_bytes, 'key': key }

In [61]:
instances = {'instances': [to_instance(v, k) for k, v in enumerate(raw_instances)]}

In [65]:
endpoint_id = '5446488022793060352'
pred = endpoint_predict(project_id, my_region, instances, endpoint_id)
pred

InvalidArgument: 400 {
    "error": "Failed to process element: 0 of 'instances' list. Error: INVALID_ARGUMENT: JSON Value: \"instances\" Type: String is not of expected type: float"
}